In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
# These data were downloaded manually from NASA's Earthdata repository
df = pd.read_csv("2022-epi-raw-data-time-series/AMP_raw.csv")
df.head()

,code,iso,country,AMP.raw.1990,AMP.raw.1991,AMP.raw.1992,AMP.raw.1993,AMP.raw.1994,AMP.raw.1995,AMP.raw.1996,...,AMP.raw.2013,AMP.raw.2014,AMP.raw.2015,AMP.raw.2016,AMP.raw.2017,AMP.raw.2018,AMP.raw.2019,AMP.raw.2020,AMP.raw.2021,AMP.raw.2022
0,4,AFG,Afghanistan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8,ALB,Albania,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,178.803804,178.803804,178.803804,178.803804,178.803804,178.803804,178.803804,178.803804,178.803804,178.803804
2,12,DZA,Algeria,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,26.558865,26.558865,26.558865,26.558865,26.558865,26.558865,26.558865,26.558865,26.558865,26.558865
3,20,AND,Andorra,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,24,AGO,Angola,1.40094,1.40094,1.40094,1.40094,1.40094,1.40094,1.40094,...,1.400940,1.400940,1.400940,1.400940,1.400940,1.400940,1.400940,1.400940,1.400940,1.400940


In [3]:
# The data from this file are not yet used, but may be helpful in future explorations
df = pd.read_csv("2022-epi-indicators-time-series/BCA_ind.csv")
df.head()

,code,iso,country,BCA.ind.1995,BCA.ind.1996,BCA.ind.1997,BCA.ind.1998,BCA.ind.1999,BCA.ind.2000,BCA.ind.2001,...,BCA.ind.2013,BCA.ind.2014,BCA.ind.2015,BCA.ind.2016,BCA.ind.2017,BCA.ind.2018,BCA.ind.2019,BCA.ind.2020,BCA.ind.2021,BCA.ind.2022
0,4,AFG,Afghanistan,15.772571,15.772571,15.772571,15.772571,15.772571,14.091805,15.148239,...,20.426463,22.263297,25.533805,31.705520,38.707909,41.037893,42.942914,42.942914,42.942914,42.942914
1,8,ALB,Albania,100.000000,100.000000,100.000000,100.000000,100.000000,44.920908,0.000000,...,94.764745,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2,12,DZA,Algeria,69.090971,69.090971,69.090971,69.090971,69.090971,64.768012,59.459235,...,53.123405,54.567219,54.078793,56.011786,59.124426,62.723414,63.867843,63.867843,63.867843,63.867843
3,20,AND,Andorra,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,24,AGO,Angola,53.762469,53.762469,53.762469,53.762469,53.762469,58.328929,59.981637,...,1.518487,4.637081,14.769993,22.203151,35.380998,44.729316,51.810194,51.810194,51.810194,51.810194


In [ ]:
df_epi = pd.DataFrame()
files_path = glob.glob("2022-epi-raw-data-time-series/*.csv")

for file in files_path:
    # Initialize a temporary dataframe to extract modern values
    df_temp = pd.read_csv(file)
    # Create the base name, which is uniquely identified by the Three-Letter Acronym (TLA) of each feature
    name_base = os.path.basename(file)[:3]

    name_2020 = f"{name_base}.raw.2020"
    if name_2020 in df_temp.columns:
        # Extract the feature for the year 2020
        df_temp_2 = df_temp[["code", "iso", "country", name_2020]].copy()
        # Rename the column to its base for clarity,
        # while losing whether the information is from 2019 or 2020
        df_temp_2.rename(columns={name_2020: name_base}, inplace=True)

    else:
        # Repeat above to extract 2019 data if 2020 data is unavailable
        # Excluding older data removes unimportant features while maintaining modernity
        name_2019 = f"{name_base}.raw.2019"
        if name_2019 in df_temp.columns:
            df_temp_2 = df_temp[["code", "iso", "country", name_2019]].copy()
            df_temp_2.rename(columns={name_2019: name_base}, inplace=True)
        else:
            continue

    # Initialize the final dataframe if it does not exist already
    if df_epi.empty:
        df_epi = df_temp_2
    else:
        # Merge the temporary dataframes based on their shared features
        df_epi = pd.merge(df_epi, df_temp_2, on = ["code", "iso", "country"], how = "outer")

# Remove values that represent missing data
invalid_values = [-9999, -8888, -7777]
df_epi.replace(invalid_values, np.nan, inplace = True)
df_epi = df_epi.fillna(df_epi.mean(numeric_only = True))
df_epi = df_epi.dropna(axis = 1)

df_epi.head(10)

,code,iso,country,AMP,APR,BCA,BHV,BLC,CDA,CDO,...,TC5,TCC,TCL,USD,UWD,VOE,WL5,WTA,WTL,WWT
0,4,AFG,Afghanistan,0.000000e+00,13.134830,0.021409,0.418988,14.917438,-0.051453,9780.0,...,85.0,39.0,0.000060,466.408266,674.481233,0.015732,0.000000,71.499301,0.000000,0.000000
1,8,ALB,Albania,1.788038e+02,9.532294,-0.111462,0.348171,0.945792,0.011736,5560.0,...,7684.0,1241.0,0.002041,39.412070,116.095860,0.009952,0.738103,56.010690,0.002602,0.018530
2,12,DZA,Algeria,2.655887e+01,1.273220,0.006700,0.596696,69.293182,0.047309,184000.0,...,84559.0,14169.0,0.011911,39.637104,127.632487,0.021838,0.000000,21.748026,0.000000,0.331200
3,20,AND,Andorra,0.000000e+00,3.480068,-0.007744,0.563142,27.617290,0.024957,458.0,...,56.0,28.0,0.000567,0.913971,1.655597,0.011440,52.151606,10938.094877,0.003112,1.000000
4,24,AGO,Angola,1.400940e+00,0.083847,0.015176,0.614766,27.555363,0.016730,27000.0,...,1400956.0,298827.0,0.004250,1453.097805,2329.179532,0.064238,0.000000,7178.013668,0.000000,0.000000
5,28,ATG,Antigua and Barbuda,2.911273e+02,9.924773,0.003106,0.339933,0.034825,0.019086,537.0,...,293.0,15.0,0.002896,46.832324,174.021747,0.001167,0.000000,22.476116,0.000000,0.157115
6,31,AZE,Azerbaijan,4.795849e+02,1.825906,0.006217,0.292404,6.282850,0.025899,38200.0,...,220.0,24.0,0.000032,132.927280,156.484142,0.012218,0.073425,41.648922,0.000467,0.039000
7,32,ARG,Argentina,1.244558e+05,9.820394,-0.006843,0.431925,31.926064,-0.002687,187000.0,...,1001565.0,202692.0,0.004856,14.052423,56.757314,0.043633,290.922640,93218.022053,0.000676,0.059236
8,36,AUS,Australia,3.021673e+06,2.879592,0.002585,0.563948,25.243400,-0.014269,381000.0,...,5331440.0,2529076.0,0.022115,2.218524,10.419513,0.039327,68.752128,101704.247872,0.000135,0.929000
9,40,AUT,Austria,0.000000e+00,7.046839,-0.092937,0.439798,3.985065,-0.006551,66600.0,...,121795.0,25990.0,0.005514,4.621392,1.874644,0.008480,0.389057,135.602407,0.000571,0.940000


In [16]:
# This code checks that no values are below -1
# -1 is the lowest possible value among all features
# There is no reasonable upper limit to place on the values
df_numeric = df_epi.select_dtypes(include=['number'])
mask = df_numeric < -1
errors_list = df_numeric[mask].stack().tolist()
print(errors_list)

[]


In [17]:
# Save the dataframe to a CSV file to be used later
df_epi.to_csv("epi_cleaned.csv", index = False)